<a href="https://colab.research.google.com/github/Trek3/stacksample-amd/blob/main/stacksample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!kaggle datasets files stackoverflow/stacksample
!kaggle datasets download stackoverflow/stacksample -p .kaggle/ -f Questions.csv
!cd .kaggle
!unzip Questions.csv.zip

name           size  creationDate         
-------------  ----  -------------------  
Answers.csv     1GB  2019-10-08 00:37:26  
Questions.csv   2GB  2019-10-08 00:37:26  
Tags.csv       62MB  2019-10-08 00:37:26  
Questions.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
unzip:  cannot find or open Questions.csv.zip, Questions.csv.zip.zip or Questions.csv.zip.ZIP.


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

with open('.kaggle/Questions.csv', 'rb') as f:
  df = pd.read_csv(f, sep=',', usecols=['Id','Body'], encoding='ISO-8859-1', index_col=0).head(100)

df

/home/trekkete/.local/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,Body
Id,
80,<p>I've written a database generation script i...
90,<p>Are there any really good tutorials explain...
120,<p>Has anyone got experience creating <strong>...
180,<p>This is something I've pseudo-solved many t...
260,<p>I have a little game written in C#. It uses...
...,...
9570,<p>I'm integrating .NET support into our C++ a...
9650,<p><br />\nI've been wanting to teach myself L...
9750,<p>I was reading Joel's book where he was sugg...


In [129]:
import random as rd
import functools

def build_hashrow_functions(n, k):
  ixs = list(range(n))
  rd.shuffle(ixs)
  return [lambda x, i=i : (x + i) % k for i in ixs]

def shingle(k, body):
  return [body[i:i+k] for i in range(len(body) // k)]

def get_all_shingles(k):
  return sorted(set(functools.reduce(lambda a, b : a+b, [shingle(k, df.iloc[i]['Body']) for i in range(len(df))])))

def sim(s1, s2):
  return sum([1 if x1 == x2 else 0 for (x1, x2) in zip(s1, s2)])/(len(s1))

def sim_shingles(s1, s2):
  pairs = list(zip(s1, s2))
  x = sum([(x1 & x2) for (x1, x2) in pairs])
  y = sum([(x1 ^ x2) for (x1, x2) in pairs])
  #y = 0
  #for (x1, x2) in pairs:
  #  print(x1, x2)
  #  if x1 == 0 and x2 == 1 or x1 == 1 and x2 == 0:
  #    y += 1

  #print(x, y)
  if x + y == 0:
    return 0.0
  return x / (x + y)

def build_characteristic_matrix(df, k = 6, shingles_per_column = None, all_shingles = None):
  if all_shingles is None:
    all_shingles = get_all_shingles(k)
  if shingles_per_column is None:
    shingles_per_column = [shingle(k, df.iloc[i]['Body']) for i in range(len(df))]

  return [[1 if shin in col_i else 0 for col_i in shingles_per_column] for shin in all_shingles]

def build_signature_matrix(df, k = 6, char_matrix = None, number_of_functions = 100):
  functions = build_hashrow_functions(number_of_functions, len(charmat))

  if char_matrix is None:
    char_matrix = build_characteristic_matrix(df, k)

  sig_matrix = [[len(char_matrix) + 1 for _ in range(len(df))] for _ in range(len(functions))]

  for i in range(len(char_matrix)):
    indexes = [f(i) for f in functions]
    for j in range(len(char_matrix[i])):
      if char_matrix[i][j] == 1:
        for l in range(len(sig_matrix)):
          sig_matrix[l][j] = min(sig_matrix[l][j], indexes[l])

  return pd.DataFrame(sig_matrix), pd.DataFrame(char_matrix)

def build_similarity_matrix(sig_df):
  return pd.DataFrame([[sim(sig_df[col1], sig_df[col2]) for col2 in sig_df] for col1 in sig_df])

def build_similarity_matrix_from_shingles(char_df):
  return pd.DataFrame([[sim_shingles(char_df[col1], char_df[col2]) for col2 in char_df] for col1 in char_df])

In [130]:
signature, characteristic = build_signature_matrix(df, k=2)

similarity = build_similarity_matrix(signature)

In [131]:
similarity

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
96,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
97,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
98,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


In [132]:
build_similarity_matrix_from_shingles(characteristic)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,0.183616,0.214854,0.184971,0.411917,0.343832,0.215743,0.287234,0.308673,0.305623,...,0.187500,0.244216,0.193642,0.310433,0.158879,0.246914,0.262887,0.188202,0.232240,0.191740
1,0.183616,1.000000,0.168269,0.160714,0.217712,0.183267,0.188235,0.137026,0.187251,0.168498,...,0.158654,0.179825,0.157895,0.171875,0.154412,0.169355,0.180258,0.162011,0.162562,0.181250
2,0.214854,0.168269,1.000000,0.225131,0.234114,0.307393,0.235897,0.229226,0.296154,0.274021,...,0.196581,0.257143,0.190955,0.255556,0.180723,0.236842,0.261044,0.204878,0.233184,0.252747
3,0.184971,0.160714,0.225131,1.000000,0.239382,0.309091,0.304054,0.217252,0.278761,0.302521,...,0.240642,0.274510,0.277027,0.265217,0.254237,0.237885,0.323383,0.236025,0.327485,0.276596
4,0.411917,0.217712,0.234114,0.239382,1.000000,0.443686,0.237736,0.343750,0.390164,0.369231,...,0.278746,0.312292,0.250965,0.387622,0.214592,0.300000,0.345638,0.201439,0.360902,0.280488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.246914,0.169355,0.236842,0.237885,0.300000,0.325260,0.291480,0.279570,0.333333,0.332237,...,0.233962,0.276978,0.229437,0.299663,0.168269,1.000000,0.337037,0.214876,0.282869,0.261468
96,0.262887,0.180258,0.261044,0.323383,0.345638,0.378277,0.287736,0.340116,0.382022,0.358885,...,0.300000,0.338583,0.280952,0.358974,0.206349,0.337037,1.000000,0.273973,0.340611,0.326531
97,0.188202,0.162011,0.204878,0.236025,0.201439,0.280851,0.271605,0.223602,0.242798,0.223485,...,0.200980,0.252294,0.239264,0.245902,0.219697,0.214876,0.273973,1.000000,0.237113,0.245161
98,0.232240,0.162562,0.233184,0.327485,0.360902,0.365145,0.307263,0.318750,0.352459,0.339695,...,0.263889,0.343750,0.270718,0.360656,0.219355,0.282869,0.340611,0.237113,1.000000,0.292398
